In [1]:
import json
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box
import time
import hashlib
import hmac
import requests
import json

In [5]:
def gen_sign(method, url, query_string=None, payload_string=None):
    sign_path = "../cred/credentials.json"

    # Read the credentials from the file
    with open(sign_path, 'r') as f:
        data = json.load(f)
        key = data['pub_key']
        secret = data['secret_key']

    t = time.time()
    m = hashlib.sha512()
    m.update((payload_string or "").encode('utf-8'))
    hashed_payload = m.hexdigest()
    s = '%s\n%s\n%s\n%s\n%s' % (method, url, query_string or "", hashed_payload, t)
    sign = hmac.new(secret.encode('utf-8'), s.encode('utf-8'), hashlib.sha512).hexdigest()
    return {'KEY': key, 'Timestamp': str(t), 'SIGN': sign}

def get_data():
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

    # url = '/account/detail'
    url = '/futures/usdt/contract_stats'
    query_param = 'contract=BTC_USDT'
    r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
    # print(r.json())
    return r.json()

In [7]:
data = get_data()
type(data)

list

In [8]:
len(data)

30

In [10]:
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/futures/usdt/positions'
query_param = ''
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign('GET', prefix + url, query_param)
headers.update(sign_headers)
r = requests.request('GET', host + prefix + url, headers=headers)
data = r.json()
print(data)


[{'value': '967.78626', 'leverage': '0', 'mode': 'single', 'realised_point': '-0.7261649325', 'contract': 'BTC_USDT', 'entry_price': '65865.3', 'mark_price': '65835.8', 'history_point': '-93.5276560279', 'realised_pnl': '0', 'close_order': None, 'size': 147, 'cross_leverage_limit': '25', 'pending_orders': 0, 'adl_ranking': 5, 'maintenance_rate': '0.004', 'unrealised_pnl': '-0.43365', 'pnl_pnl': '0', 'pnl_fee': '0', 'pnl_fund': '0', 'user': 15080783, 'leverage_max': '125', 'history_pnl': '-173.112725670776', 'risk_limit': '1000000', 'margin': '143.784216064116', 'last_close_pnl': '-4.774686065842', 'liq_price': '56351.74', 'update_time': 1713757894, 'update_id': 434, 'initial_margin': '0', 'maintenance_margin': '0', 'open_time': 1713757894, 'trade_max_size': '0'}, {'value': '0', 'leverage': '0', 'mode': 'single', 'realised_point': '0', 'contract': 'ETH_USDT', 'entry_price': '0', 'mark_price': '3201.55', 'history_point': '-51.91307841', 'realised_pnl': '0', 'close_order': None, 'size': 0

In [24]:
# Show all columns
[v for v in data[0]]

['value',
 'leverage',
 'mode',
 'realised_point',
 'contract',
 'entry_price',
 'mark_price',
 'history_point',
 'realised_pnl',
 'close_order',
 'size',
 'cross_leverage_limit',
 'pending_orders',
 'adl_ranking',
 'maintenance_rate',
 'unrealised_pnl',
 'pnl_pnl',
 'pnl_fee',
 'pnl_fund',
 'user',
 'leverage_max',
 'history_pnl',
 'risk_limit',
 'margin',
 'last_close_pnl',
 'liq_price',
 'update_time',
 'update_id',
 'initial_margin',
 'maintenance_margin',
 'open_time',
 'trade_max_size']

In [23]:
# Show position which value is not 0 in contract real time
[(v['contract'], v['value'], v['realised_pnl'], v['unrealised_pnl']) for v in data if v['value'] != '0']

[('BTC_USDT', '967.78626', '0', '-0.43365'),
 ('DOGE_USDT', '269.2374', '-0.040147635', '1.5865')]

In [21]:
[v for v in data if v['value'] != '0']


[{'value': '967.78626',
  'leverage': '0',
  'mode': 'single',
  'realised_point': '-0.7261649325',
  'contract': 'BTC_USDT',
  'entry_price': '65865.3',
  'mark_price': '65835.8',
  'history_point': '-93.5276560279',
  'realised_pnl': '0',
  'close_order': None,
  'size': 147,
  'cross_leverage_limit': '25',
  'pending_orders': 0,
  'adl_ranking': 5,
  'maintenance_rate': '0.004',
  'unrealised_pnl': '-0.43365',
  'pnl_pnl': '0',
  'pnl_fee': '0',
  'pnl_fund': '0',
  'user': 15080783,
  'leverage_max': '125',
  'history_pnl': '-173.112725670776',
  'risk_limit': '1000000',
  'margin': '143.784216064116',
  'last_close_pnl': '-4.774686065842',
  'liq_price': '56351.74',
  'update_time': 1713757894,
  'update_id': 434,
  'initial_margin': '0',
  'maintenance_margin': '0',
  'open_time': 1713757894,
  'trade_max_size': '0'},
 {'value': '269.2374',
  'leverage': '0',
  'mode': 'single',
  'realised_point': '-0.16059054',
  'contract': 'DOGE_USDT',
  'entry_price': '0.16027',
  'mark_pric

In [28]:
# Show contract history:
# # coding: utf-8
# import requests
# import time
# import hashlib
# import hmac



host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/futures/usdt/my_trades'
query_param = ''
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign('GET', prefix + url, query_param)
headers.update(sign_headers)
r = requests.request('GET', host + prefix + url, headers=headers)
print(r.json())


# package above as a function
def get_history_contract(url = '/futures/usdt/my_trades'):
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    query_param = ''
    sign_headers = gen_sign('GET', prefix + url, query_param)
    headers.update(sign_headers)
    r = requests.request('GET', host + prefix + url, headers=headers)
    return r.json()

[{'id': 78023643, 'contract': 'DOGE_USDT', 'create_time': 1713760712.9959, 'size': 167, 'price': '0.16027', 'order_id': '457658602279', 'fee': '0.040147635', 'point_fee': '0.16059054', 'role': 'taker', 'text': 'app'}, {'id': 302641564, 'contract': 'BTC_USDT', 'create_time': 1713757894.5539, 'size': 147, 'price': '65865.3', 'order_id': '457634417355', 'fee': '0', 'point_fee': '0.7261649325', 'role': 'taker', 'text': 'app'}, {'id': 302410369, 'contract': 'BTC_USDT', 'create_time': 1713715584.0252, 'size': -58, 'price': '64476.5', 'order_id': '457352453073', 'fee': '0.03739637', 'point_fee': '0.243076405', 'role': 'taker', 'text': 'ao-69040532'}, {'id': 302340670, 'contract': 'BTC_USDT', 'create_time': 1713698474.1737, 'size': 58, 'price': '65282', 'order_id': '457232247281', 'fee': '0.03786356', 'point_fee': '0', 'role': 'maker', 'text': 'app'}, {'id': 302336140, 'contract': 'BTC_USDT', 'create_time': 1713698212.0451, 'size': -171, 'price': '65199.8', 'order_id': '457231319720', 'fee': '

In [30]:
history_data = get_history_contract()
len(history_data)

100

In [32]:
history_data[0]

{'id': 78023643,
 'contract': 'DOGE_USDT',
 'create_time': 1713760712.9959,
 'size': 167,
 'price': '0.16027',
 'order_id': '457658602279',
 'fee': '0.040147635',
 'point_fee': '0.16059054',
 'role': 'taker',
 'text': 'app'}

In [33]:
# 查询个人成交记录(时间区间)
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/futures/usdt/my_trades_timerange'
query_param = ''
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign('GET', prefix + url, query_param)
headers.update(sign_headers)
r = requests.request('GET', host + prefix + url, headers=headers)
print(r.json())

[{'price': '0.16027', 'text': 'app', 'fee': '0.040147635', 'create_time': 1713760712.9959, 'point_fee': '0.16059054', 'trade_id': '78023643', 'contract': 'DOGE_USDT', 'role': 'taker', 'order_id': '457658602279', 'size': 167, 'biz_info': '-', 'amend_text': '-'}, {'price': '65865.3', 'text': 'app', 'fee': '0', 'create_time': 1713757894.5539, 'point_fee': '0.7261649325', 'trade_id': '302641564', 'contract': 'BTC_USDT', 'role': 'taker', 'order_id': '457634417355', 'size': 147, 'biz_info': '-', 'amend_text': '-'}, {'price': '64476.5', 'text': 'ao-69040532', 'fee': '0.03739637', 'create_time': 1713715584.0252, 'point_fee': '0.243076405', 'trade_id': '302410369', 'contract': 'BTC_USDT', 'role': 'taker', 'order_id': '457352453073', 'size': -58, 'biz_info': '-', 'amend_text': '-'}, {'price': '65282', 'text': 'app', 'fee': '0.03786356', 'create_time': 1713698474.1737, 'point_fee': '0', 'trade_id': '302340670', 'contract': 'BTC_USDT', 'role': 'maker', 'order_id': '457232247281', 'size': 58, 'biz_

In [35]:
# 平倉歷史
host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/futures/usdt/position_close'
query_param = ''
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign('GET', prefix + url, query_param)
headers.update(sign_headers)
r = requests.request('GET', host + prefix + url, headers=headers)
print(r.json())
print(len(r.json()))

[{'contract': 'BTC_USDT', 'text': 'ao-69040532', 'long_price': '65282', 'pnl': '-4.774686065842', 'pnl_pnl': '-4.6719', 'pnl_fee': '-0.07525993', 'pnl_fund': '-0.027526135842', 'accum_size': '58', 'time': 1713715584, 'short_price': '64476.5', 'side': 'long', 'max_size': '58', 'first_open_time': 1713698474}, {'contract': 'BTC_USDT', 'text': 'ao-69035380', 'long_price': '64884.3', 'pnl': '5.1837553548', 'pnl_pnl': '5.39505', 'pnl_fee': '-0.2112946452', 'pnl_fund': '0', 'accum_size': '171', 'time': 1713698212, 'short_price': '65199.8', 'side': 'long', 'max_size': '171', 'first_open_time': 1713693807}, {'contract': 'BTC_USDT', 'text': 'web', 'long_price': '65242', 'pnl': '-5.47167291', 'pnl_pnl': '-5.2569', 'pnl_fee': '-0.21477291', 'pnl_fund': '0', 'accum_size': '165', 'time': 1713684602, 'short_price': '64923.4', 'side': 'long', 'max_size': '165', 'first_open_time': 1713670872}, {'contract': 'DOGE_USDT', 'text': 'ao-68968129', 'long_price': '0.15846', 'pnl': '3.6759366', 'pnl_pnl': '3.79

In [37]:
# Getting the commission history
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/rebate/agency/commission_history'
query_param = ''
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign('GET', prefix + url, query_param)
headers.update(sign_headers)
r = requests.request('GET', host + prefix + url, headers=headers)
print(r.json())


{'total': 80, 'list': [{'user_id': 15080783, 'commission_time': 1713334651, 'commission_amount': '0.0380599128', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commission_time': 1713338273, 'commission_amount': '0.0383083786', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commission_time': 1713341826, 'commission_amount': '0.0301810320', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commission_time': 1713341833, 'commission_amount': '0.0302888160', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commission_time': 1713341859, 'commission_amount': '0.0079603268', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commission_time': 1713341859, 'commission_amount': '0.0194313225', 'commission_asset': 'USDT', 'source': 'FUTURES', 'group_name': ''}, {'user_id': 15080783, 'commis